In [1]:
# Funny installation method for installing into this notebook in a docker container
# In real use this should be "pip install o11y"
import platform

# Detect the architecture
arch = platform.machine()

# Determine the appropriate wheel file based on the architecture
if arch == 'x86_64':
    wheel_file = './dist/*linux_x86_64.whl'
elif arch == 'aarch64' or arch == 'arm64':
    wheel_file = './dist/*linux_aarch64.whl'
else:
    raise ValueError(f"Unsupported architecture: {arch}")


!pip install {wheel_file} --force-reinstall

Processing ./dist/o11y-0.1.0-py3-none-linux_aarch64.whl
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.6/62.6 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.8/163.8 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 136.6/136.6 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.1/121.1 kB 21.9 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.2.1
    Uninstalling urllib3-2.2.1:
      Successfully uninstalled urllib3-2.2.1
  Attempting uninstall: idna
    Found existing installation: idna 3.7
    Uninstalling idna-3.7:
      Successfully uninstalled idna-3.7
  Attempting uninstall: charset-normalizer
    Found existing installation: charset-normalizer 3.3.2
    Uninstalling charset-normalizer-3.3.2:
      Successfully uninstalled charset-normalizer-3.3.2
  Attempting uninstall: certifi
    Found existing 

In [2]:
import o11y

Go plugin launched successfully


In [3]:
import os
# Verify we have credentials set in environment
# In principle it should be possible to do "o11y login" to set this from the command line
print(os.environ['GF_AI_TRAINING_CREDS'])

83bcaff6228b39bbe431af5e19fb4368e2a03dd3@ai-training-api:8000


In [4]:
import random 

epochs = 10
lr = 0.01

run = o11y.init(
    # Set the project where this run will be logged
    project="my-awesome-project",
    # Track hyperparameters and run metadata
    metadata={
        "config": {
            "learning_rate": lr,
            "epochs": epochs,
            "feed_forward": {
                "activation": "swiglu",
                "d_model": 1024,
            }
        },
        "git_commit": "some_hash_goes_here",
    },
)

offset = random.random() / 5
print(f"lr: {lr}")

# simulating a training run
for epoch in range(2, epochs):
    acc = 1 - 2**-epoch - random.random() / epoch - offset
    loss = 2**-epoch + random.random() / epoch + offset
    print(f"epoch={epoch}, accuracy={acc}, loss={loss}")
    o11y.log({"accuracy": acc, "loss": loss})
    if epoch == epochs:
        o11y.update({
            "final_accuracy": acc,
            "final_loss": loss
        })

lr: 0.01
epoch=2, accuracy=0.3619442396391509, loss=0.4670409088511358
epoch=3, accuracy=0.6796477868289498, loss=0.3582889067034011
epoch=4, accuracy=0.7605710182415426, loss=0.22021099963063268
epoch=5, accuracy=0.7995029906952056, loss=0.30807063140012325
epoch=6, accuracy=0.7141078943823662, loss=0.26216797631156985
epoch=7, accuracy=0.7195124522352594, loss=0.18917365674531686
epoch=8, accuracy=0.7879089093229549, loss=0.19417962404846562
epoch=9, accuracy=0.8470992428897597, loss=0.15444473279346893
